### Project : Analysing Music Records Sales using SQL
Chinook is a fictional music record store based out of the USA. The store offers a wide range of music genres, from classic rock and jazz to modern indie and hip-hop. With a passion for music and a commitment to providing exceptional customer
experiences, Chinook aims to be the go-to destination for music lovers.

### To achieve so, they have the following business objectives:
1. Establish Chinook as the premier destination for music records.
2. Provide a diverse selection of music genres to cater to a wide range of customer preferences.
3. Deliver excellent customer service to enhance the shopping experience.
4. Build a loyal customer base and foster long-term relationships through engagement and promotional events.

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
database = 'chinook.db'

In [2]:
 conn = sqlite3.connect(database)
tables = pd.read_sql("""SELECT *
 FROM sqlite_master
 WHERE type='table';
 """, conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


### Segment 1: Database - Tables, Columns, Relationships
Identify the tables in the database and their respective columns.
Determine the number of records in each table within the schema.
Identify and handle any missing or inconsistent values in the dataset.
Analyse the data types of the columns in each table to ensure they are appropriate for the stor

In [3]:
query1 = pd.read_sql("""
pragma table_info('customer');
""", conn)
query1

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,INTEGER,1,None,1
1,1,first_name,NVARCHAR(40),1,None,0
2,2,last_name,NVARCHAR(20),1,None,0
3,3,company,NVARCHAR(80),0,None,0
4,4,address,NVARCHAR(70),0,None,0
5,5,city,NVARCHAR(40),0,None,0
6,6,state,NVARCHAR(40),0,None,0
7,7,country,NVARCHAR(40),0,None,0
8,8,postal_code,NVARCHAR(10),0,None,0
9,9,phone,NVARCHAR(24),0,None,0


### Segment 2: Track/Album Analysis
How many tracks does Chinook store have in total

In [9]:
query2 = pd.read_sql("""
SELECT count(Distinct Name) as Total_No_Of_Track_Is
FROM Track;
""", conn)
query2


,Total_No_Of_Track_Is
0,3257


### Which tracks are the most popular among customers?

In [11]:

query3 = pd.read_sql("""
Select t.track_Id, t.Name, count(il.Track_Id) as No_of_sales
from Track t
inner join Invoice_Line il
on t.Track_Id = il.Track_Id
group by 1 , 2
order by 3 desc
limit 10
""", conn)
query3


,track_id,name,No_of_sales
0,3336,War Pigs,31
1,1489,Are You Experienced?,14
2,1495,Highway Chile,14
3,6,Put The Finger On You,13
4,1487,Third Stone From The Sun,13
5,1490,Hey Joe,13
6,1483,Love Or Confusion,12
7,2558,Radio/Video,12
8,1129,Dead And Broken,11
9,1479,Foxy Lady,11


### What are the top-selling albums in the Chinook store? 

In [14]:

query4 = pd.read_sql("""
SELECT
 a.Album_Id,
 a.Title AS AlbumTitle,
 COUNT(il.Invoice_Line_Id) AS NumTracksSold
FROM
 Album a
JOIN
 Track t ON a.Album_Id = t.Album_Id
JOIN
 Invoice_Line il ON t.Track_Id = il.Track_Id
GROUP BY
 a.Album_Id, a.Title
ORDER BY
 NumTracksSold DESC
LIMIT 10;
""",conn)
query4

,album_id,AlbumTitle,NumTracksSold
0,120,Are You Experienced?,187
1,88,Faceless,96
2,207,Mezmerize,93
3,119,Get Born,90
4,214,The Doors,83
5,5,Big Ones,80
6,185,Greatest Hits I,80
7,215,The Police Greatest Hits,80
8,163,From The Muddy Banks Of The Wishkah [live],78
9,221,My Generation - The Very Best Of The Who,76


### Which artist or album has the highest number of tracks in the store

In [18]:
query5 = pd.read_sql("""
SELECT
 a.Artist_Id,
 a.Name AS ArtistName,
 al.Album_Id,
 al.Title AS AlbumTitle,
 COUNT(t.Track_Id) AS NumTracks
FROM
 Artist a
LEFT JOIN
 Album al ON a.Artist_Id = al.Artist_Id
LEFT JOIN
 Track t ON al.Album_Id = t.Album_Id
GROUP BY
 a.Artist_Id, a.Name, al.Album_Id, al.Title
ORDER BY
 NumTracks DESC
LIMIT 1;
""", conn)
query5

,artist_id,ArtistName,album_id,AlbumTitle,NumTracks
0,100,Lenny Kravitz,141,Greatest Hits,57


### Determine the number of tracks for each album and calculate the total duration of each album. Identify the albums that have the highest average track duration.

In [23]:

query6 = pd.read_sql("""
select 
a.album_id, a.title as Album_Title,count(t.track_id),
sum(t.Milliseconds)/60000 as DurationOfAlbum,
round(AVG(t.Milliseconds),2)/60000 AS AverageTrackDurationMilliseconds
from album a 
left join track t
on a.album_id = t.album_id
group by 1,2
order by 5 desc
""",conn)
query6


,album_id,Album_Title,count(t.track_id),DurationOfAlbum,AverageTrackDurationMilliseconds
0,253,"Battlestar Galactica (Classic), Season 1",24,1170,48.759572
1,227,"Battlestar Galactica, Season 3",19,879,46.304422
2,229,"Lost, Season 3",26,1177,45.298450
3,231,"Lost, Season 2",24,1054,43.951133
4,226,Battlestar Galactica: The Story So Far,1,43,43.704167
...,...,...,...,...,...
342,328,"Charpentier: Divertissements, Airs & Concerts",1,1,1.837767
343,314,English Renaissance,2,3,1.691350
344,318,SCRIABIN: Vers la flamme,1,1,1.688217
345,345,Monteverdi: L'Orfeo,1,1,1.110650


### How does the genre distribution vary among albums?

In [29]:
query7 = pd.read_sql("""
SELECT
 a.Album_Id,
 a.Title AS AlbumTitle,
 COUNT(DISTINCT t.Genre_Id) AS NumberOfGenres,
 GROUP_CONCAT(DISTINCT g.Name) AS GenresList
FROM
 Album a
LEFT JOIN
 Track t ON a.Album_Id = t.Album_Id
LEFT JOIN
 Genre g ON t.Genre_Id = g.Genre_Id
GROUP BY
 a.Album_Id, a.Title
ORDER BY
 NumberOfGenres DESC;
""",conn)
query7


,album_id,AlbumTitle,NumberOfGenres,GenresList
0,141,Greatest Hits,3,"Rock,Reggae,Metal"
1,227,"Battlestar Galactica, Season 3",3,"TV Shows,Science Fiction,Sci Fi & Fantasy"
2,73,Unplugged,2,"Blues,Latin"
3,102,Live After Death,2,"Heavy Metal,Metal"
4,109,Rock In Rio [cd2],2,"Rock,Metal"
...,...,...,...,...
342,343,Respighi:Pines of Rome,1,Classical
343,344,Schubert: The Late String Quartets & String Qu...,1,Classical
344,345,Monteverdi: L'Orfeo,1,Classical
345,346,Mozart: Chamber Music,1,Classical


In [31]:

query8 = pd.read_sql("""
select g.name,count(a.album_id) as NoofAlbum
from genre g
join track t
on t.genre_id = g.genre_id
join album a
on t.album_id = a.album_id
group by 1
order by 2 desc
""", conn)
query8


,name,NoofAlbum
0,Rock,1297
1,Latin,579
2,Metal,374
3,Alternative & Punk,332
4,Jazz,130
5,TV Shows,93
6,Blues,81
7,Classical,74
8,Drama,64
9,R&B/Soul,61


### Segment 3: Customer Analysis
Analyse customer demographics, purchase history, and preferences.

In [32]:
query9 = pd.read_sql("""
select *
from customer
""", conn)
query9

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
5,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
6,7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
7,8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
8,9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
9,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


### How many unique customers are there in the Chinook store?


In [34]:

quer10 = pd.read_sql("""
SELECT COUNT(DISTINCT Customer_Id) AS UniqueCustomersCount
 FROM Customer;
""",conn)
quer10

,UniqueCustomersCount
0,59


#### Identify the average number of tracks purchased per customer for each genre

In [36]:

query11 = pd.read_sql("""
SELECT g.Name AS GenreName,
 COUNT(il.Track_Id) / COUNT(DISTINCT c.Customer_Id) AS AvgTracksPurchasedPerCustomer
FROM Genre g
JOIN Track t ON g.Genre_Id = t.Genre_Id
JOIN Invoice_Line il ON t.Track_Id = il.Track_Id
JOIN Invoice i ON il.Invoice_Id = i.Invoice_Id
JOIN Customer c ON i.Customer_Id = c.Customer_Id
GROUP BY g.Name
order by 2 desc
limit 10;
""",conn)
query11


,GenreName,AvgTracksPurchasedPerCustomer
0,Rock,44
1,Metal,10
2,Alternative & Punk,8
3,R&B/Soul,3
4,Latin,3
5,Blues,3
6,Jazz,2
7,Hip Hop/Rap,2
8,Electronica/Dance,2
9,Easy Listening,2


### What is the average purchase value per customer?

In [39]:

query12 = pd.read_sql("""
SELECT round(AVG(total_purchase),2) AS AveragePurchaseValuePerCustomer
FROM (
 SELECT c.Customer_Id, SUM(il.Unit_Price) AS total_purchase
 FROM Customer c
 JOIN Invoice i ON c.Customer_Id = i.Customer_Id
 JOIN Invoice_Line il ON i.Invoice_Id = il.Invoice_Id
 GROUP BY c.Customer_Id
) AS customer_purchases;
""",conn)
query12

,AveragePurchaseValuePerCustomer
0,79.82


### Which customers have made the highest number of purchases?
 

In [45]:
query13 = pd.read_sql("""
SELECT c.Customer_Id, c.First_Name, c.Last_Name, COUNT(i.Invoice_Id) AS PurchaseCount
FROM Customer c
JOIN Invoice i ON c.Customer_Id = i.Customer_Id
GROUP BY c.Customer_Id, c.First_Name, c.Last_Name
ORDER BY PurchaseCount DESC;
""",conn)
query13


,customer_id,first_name,last_name,PurchaseCount
0,5,František,Wichterlová,18
1,35,Madalena,Sampaio,16
2,13,Fernanda,Ramos,15
3,1,Luís,Gonçalves,13
4,30,Edward,Francis,13
5,34,João,Fernandes,13
6,46,Hugh,O'Reilly,13
7,57,Luis,Rojas,13
8,58,Manoj,Pareek,13
9,6,Helena,Holý,12


### Segment 4: Support analysis
What is the total sales revenue generated by each sales support agent?

In [54]:

query14 = pd.read_sql("""
SELECT e.Employee_Id, e.first_name||" "||e.last_name as FullName, SUM(il.Unit_Price) AS TotalSalesRevenue
FROM Employee e
JOIN Customer c ON e.Employee_Id = c.Support_Rep_Id
JOIN Invoice i ON c.Customer_Id = i.Customer_Id
JOIN Invoice_Line il ON i.Invoice_Id = il.Invoice_Id
GROUP BY e.Employee_Id, e.First_Name, e.Last_Name;
""",conn)
query14


,employee_id,FullName,TotalSalesRevenue
0,3,Jane Peacock,1731.51
1,4,Margaret Park,1584.00
2,5,Steve Johnson,1393.92


### What is the average order value for each sales support agent?

In [60]:

query15 = pd.read_sql("""
SELECT e.Employee_Id, e.first_name||" "||e.last_name as FullName, 
 round(AVG(i.Total),2) AS AverageOrderValue
FROM Employee e
JOIN Customer c ON e.Employee_Id = c.Support_Rep_Id
JOIN Invoice i ON c.Customer_Id = i.Customer_Id
GROUP BY e.Employee_Id, e.First_Name, e.Last_Name;
""",conn)
query15


,employee_id,FullName,AverageOrderValue
0,3,Jane Peacock,8.17
1,4,Margaret Park,7.40
2,5,Steve Johnson,7.41


### Is there any significant difference in sales performance between agents reporting to different managers?

In [64]:
query16 = pd.read_sql("""
SELECT e2.Employee_Id AS Manager_Id, e2.First_Name||" "|| e2.last_name AS ManagerName,
 e1.Employee_Id AS Agent_Id, e1.First_Name||" "|| e1.last_name AS AgentName,
 AVG(i.Total) AS AverageOrderValue
FROM Employee e1
JOIN Employee e2 ON e1.Reports_To = e2.Employee_Id
JOIN Customer c ON e1.Employee_Id = c.Support_Rep_Id
JOIN Invoice i ON c.Customer_Id = i.Customer_Id
GROUP BY e2.Employee_Id, e2.First_Name, e2.Last_Name, e1.Employee_Id, e1.First_Name, e1.Last_Name;
""",conn)
query16

,Manager_Id,ManagerName,Agent_Id,AgentName,AverageOrderValue
0,2,Nancy Edwards,3,Jane Peacock,8.167500
1,2,Nancy Edwards,4,Margaret Park,7.401869
2,2,Nancy Edwards,5,Steve Johnson,7.414468


### How does the hiring date of sales support agents impact their sales performance? 

In [68]:
query17 = pd.read_sql("""
SELECT e.Employee_Id,e.first_name ||" "||e.last_name as FullName, e.Hire_Date,
 AVG(i.Total) AS AverageOrderValue
FROM Employee e
JOIN Customer c ON e.Employee_Id = c.Support_Rep_Id
JOIN Invoice i ON c.Customer_Id = i.Customer_Id
GROUP BY e.Employee_Id, e.First_Name, e.Last_Name, e.Hire_Date;
""",conn)
query17

,employee_id,FullName,hire_date,AverageOrderValue
0,3,Jane Peacock,2017-04-01 00:00:00,8.167500
1,4,Margaret Park,2017-05-03 00:00:00,7.401869
2,5,Steve Johnson,2017-10-17 00:00:00,7.414468


### Segment 5: Sales Analysis
What is the total revenue generated by the Chinook store?


In [71]:
query18 = pd.read_sql("""
select sum(unit_price*quantity) as TotalRevenue
from invoice_line
""",conn)
query18


,TotalRevenue
0,4709.43


### How does the revenue vary across different countries?


In [78]:
query19 = pd.read_sql("""
select i.billing_country as Country,sum(il.unit_price*il.quantity) as Revenue
from invoice_line il
right join invoice i
on il.invoice_id = i.invoice_id
group by 1
""",conn)
query19


,Country,Revenue
0,Argentina,39.60
1,Australia,81.18
2,Austria,69.30
3,Belgium,60.39
4,Brazil,427.68
5,Canada,535.59
6,Chile,97.02
7,Czech Republic,273.24
8,Denmark,37.62
9,Finland,79.20


### Calculate the percentage of total sales contributed by each genre in the USA and identify the best-selling genres and artists.

In [79]:
query20 = pd.read_sql("""
select g.name as Genre,
a.name as Artist,
SUM(il.Quantity * il.Unit_Price) AS TotalSales,
 ROUND(SUM(il.Quantity * il.Unit_Price) / (SELECT SUM(Quantity * Unit_Price) FROM Invoice_Line)* 100, 2) AS PercentageOfTotalSales
from genre g
inner join track t
on g.genre_id = t.genre_id
inner join invoice_line il
on t.track_id = il.track_id
inner join invoice i
on il.invoice_id = i.invoice_id
inner join album al
on al.album_id = t.album_id
inner join artist a
on a.artist_id = al.artist_id
where i.billing_country= 'USA'
group by 1,2
order by 3 desc
""",conn)
query20


,Genre,Artist,TotalSales,PercentageOfTotalSales
0,Rock,Van Halen,42.57,0.90
1,Rock,The Rolling Stones,36.63,0.78
2,Rock,Nirvana,34.65,0.74
3,Alternative & Punk,Green Day,31.68,0.67
4,Alternative & Punk,R.E.M.,31.68,0.67
...,...,...,...,...
111,Metal,Bruce Dickinson,0.99,0.02
112,Rock,Faith No More,0.99,0.02
113,Soundtrack,Gilberto Gil,0.99,0.02
114,Soundtrack,Passengers,0.99,0.02


### Evaluate sales performance over time to identify seasonal trends and peak periods. 

In [83]:
query21 = pd.read_sql("""
select date(i.invoice_date) as Date, sum(il.quantity*il.unit_price) as TotalSales
from invoice i
inner join invoice_line il
on i.invoice_id = il.invoice_id
group by 1
""",conn)
query21


,Date,TotalSales
0,2017-01-03,25.74
1,2017-01-05,1.98
2,2017-01-06,7.92
3,2017-01-07,16.83
4,2017-01-10,1.98
...,...,...
498,2020-12-20,3.96
499,2020-12-21,6.93
500,2020-12-27,13.86
501,2020-12-29,8.91


### Determine the average transaction value and customer lifetime value to gauge business profitability.


In [86]:
query22 = pd.read_sql("""
select c.customer_id, avg(i.total) as AvgTransaction, sum(i.total) as LifetimeValue
from customer c
join invoice i
on c.customer_id = i.customer_id
group by 1;
""", conn)
query22

,customer_id,AvgTransaction,LifetimeValue
0,1,8.376923,108.90
1,2,7.470000,82.17
2,3,11.110000,99.99
3,4,8.030000,72.27
4,5,8.030000,144.54
5,6,10.725000,128.70
6,7,7.700000,69.30
7,8,8.627143,60.39
8,9,3.762000,37.62
9,10,5.032500,60.39


### Segment 6: SQL Concepts
Use subqueries to find the top-selling tracks in the USA and identify their associated genres.


In [88]:
query23 = pd.read_sql("""
SELECT
 t.Name AS TrackName,
 g.Name AS GenreName,
 (SELECT COUNT(*) FROM Invoice_Line il
 JOIN Invoice i ON il.Invoice_Id = i.Invoice_Id
 JOIN Customer c ON i.Customer_Id = c.Customer_Id
 WHERE il.Track_Id = t.Track_Id AND c.Country = 'USA') AS TotalSales
FROM
 Track t
 JOIN Genre g ON t.Genre_Id = g.Genre_Id
 where totalsales != 0
ORDER BY
 TotalSales DESC
""", conn)
query23


,TrackName,GenreName,TotalSales
0,War Pigs,Alternative,6
1,You Know I'm No Good (feat. Ghostface Killah),R&B/Soul,5
2,Night Of The Long Knives,Rock,4
3,Evil Woman,Metal,4
4,Highway Chile,Rock,4
...,...,...,...
785,October Song,Pop,1
786,What Is It About Men,Pop,1
787,Help Yourself,Pop,1
788,Amy Amy Amy (Outro),Pop,1


### Implement joins to retrieve customer information along with their preferred genres and country of origin.

In [90]:

query24 = pd.read_sql("""
SELECT c.Customer_Id, c.First_Name, c.Last_Name, c.Country, g.Name AS PreferredGenre
FROM
 Customer AS c
 LEFT JOIN Invoice AS i ON c.Customer_Id = i.Customer_Id
 LEFT JOIN Invoice_Line AS il ON i.Invoice_Id = il.Invoice_Id
 LEFT JOIN Track AS t ON il.Track_Id = t.Track_Id
 LEFT JOIN Genre AS g ON t.Genre_Id = g.Genre_Id
 
GROUP BY 1;
""", conn)
query24

,customer_id,first_name,last_name,country,PreferredGenre
0,1,Luís,Gonçalves,Brazil,Rock
1,2,Leonie,Köhler,Germany,Rock
2,3,François,Tremblay,Canada,Rock
3,4,Bjørn,Hansen,Norway,Rock
4,5,František,Wichterlová,Czech Republic,Alternative & Punk
5,6,Helena,Holý,Czech Republic,Rock
6,7,Astrid,Gruber,Austria,Rock
7,8,Daan,Peeters,Belgium,R&B/Soul
8,9,Kara,Nielsen,Denmark,Electronica/Dance
9,10,Eduardo,Martins,Brazil,Alternative & Punk


### Apply window functions to rank genres based on their sales performance in the USA.


In [96]:

query25 = pd.read_sql("""
select rank() over(order by TotalQuantitySale desc) as Rank, *
from (select g.name as GenreName,count(il.quantity) as TotalQuantitySale
from invoice_line il
join track t on t.track_id = il.track_id
join genre g on g.genre_id = t.genre_id
join invoice i on i.invoice_id = il.invoice_id
where i.billing_country = 'USA'
group by g.name
);
""", conn)
query25

,Rank,GenreName,TotalQuantitySale
0,1,Rock,561
1,2,Alternative & Punk,130
2,3,Metal,124
3,4,R&B/Soul,53
4,5,Blues,36
5,6,Alternative,35
6,7,Latin,22
7,7,Pop,22
8,9,Hip Hop/Rap,20
9,10,Jazz,14


### Create stored procedures to calculate the average revenue per sale for each genre in the USA.


In [100]:

query26 = pd.read_sql("""
select g.name as Genre, avg(il.quantity*il.unit_price) as AvgRevenue
from invoice_line il
join track t on t.track_id = il.track_id
join genre g on g.genre_id = t.genre_id
join invoice i on i.invoice_id = il.invoice_id
where i.billing_country = 'USA'
group by g.name
order by 2 desc;
""", conn)
query26

,Genre,AvgRevenue
0,Rock,0.99
1,R&B/Soul,0.99
2,Reggae,0.99
3,Jazz,0.99
4,Easy Listening,0.99
5,TV Shows,0.99
6,Soundtrack,0.99
7,Electronica/Dance,0.99
8,Classical,0.99
9,Hip Hop/Rap,0.99


### Segment 7: Market Expansion and Advertising Strategy
Recommend the three albums from the new record label that should be prioritised for advertising and promotion in the USA based on genre sales analysis.


In [105]:

query27 = pd.read_sql("""
SELECT
 a.Title AS AlbumTitle,
 g.Name AS GenreName,
 SUM(il.Quantity) AS TotalSales
FROM
 Album AS a
 JOIN Track AS t ON a.Album_Id = t.Album_Id
 JOIN Genre AS g ON t.Genre_Id = g.Genre_Id
 JOIN Invoice_Line AS il ON t.Track_Id = il.Track_Id
 JOIN Invoice AS i ON il.Invoice_Id = i.Invoice_Id
 JOIN Customer AS c ON i.Customer_Id = c.Customer_Id
WHERE
 c.Country = 'USA'
GROUP BY
 a.Album_Id
ORDER BY 3 desc;
""", conn)
query27

,AlbumTitle,GenreName,TotalSales
0,From The Muddy Banks Of The Wishkah [live],Rock,28
1,Are You Experienced?,Rock,28
2,The Doors,Rock,27
3,Seek And Shall Find: More Of The Best (1963-1981),R&B/Soul,26
4,"Hot Rocks, 1964-1971 (Disc 1)",Rock,25
...,...,...,...
167,Body Count,Alternative & Punk,1
168,Alcohol Fueled Brewtality Live! [Disc 1],Metal,1
169,Out Of Exile,Alternative & Punk,1
170,Restless and Wild,Rock,1


### Identify high-value customers and develop personalised marketing strategies to nurture customer loyalty.

In [107]:

query28 = pd.read_sql("""
SELECT
 c.Customer_Id,
 c.First_Name ||" "||c.Last_Name as FullName,
 SUM(il.Unit_Price * il.Quantity) AS TotalRevenue,
 COUNT(DISTINCT i.Invoice_Id) AS TotalPurchases,
 AVG(il.Unit_price * il.Quantity) AS AverageOrderValue
FROM
 Customer AS c
 JOIN Invoice AS i ON c.Customer_Id = i.Customer_Id
 JOIN Invoice_Line AS il ON i.Invoice_Id = il.Invoice_Id
GROUP BY
 c.Customer_Id
HAVING
 TotalRevenue > 10
ORDER BY
 TotalRevenue DESC;
""", conn)
query28


,customer_id,FullName,TotalRevenue,TotalPurchases,AverageOrderValue
0,5,František Wichterlová,144.54,18,0.99
1,6,Helena Holý,128.70,12,0.99
2,46,Hugh O'Reilly,114.84,13,0.99
3,58,Manoj Pareek,111.87,13,0.99
4,1,Luís Gonçalves,108.90,13,0.99
5,13,Fernanda Ramos,106.92,15,0.99
6,34,João Fernandes,102.96,13,0.99
7,42,Wyatt Girard,99.99,11,0.99
8,3,François Tremblay,99.99,9,0.99
9,53,Phil Hughes,98.01,11,0.99


### Analyse the potential for expanding sales of different genres in other countries based on their sales performance in the USA.

In [109]:

query29 = pd.read_sql("""
SELECT
 g.Name AS GenreName,
 SUM(il.Quantity) AS TotalSales
FROM
 Genre AS g
 JOIN Track AS t ON g.Genre_Id = t.Genre_Id
 JOIN Invoice_Line AS il ON t.Track_Id = il.Track_Id
 JOIN Invoice AS i ON il.Invoice_Id = i.Invoice_Id
 JOIN Customer AS c ON i.Customer_Id = c.Customer_Id
WHERE
 c.Country = 'USA'
GROUP BY
 g.Genre_Id
""", conn)
query29

,GenreName,TotalSales
0,Rock,561
1,Jazz,14
2,Metal,124
3,Alternative & Punk,130
4,Blues,36
5,Latin,22
6,Reggae,6
7,Pop,22
8,Soundtrack,2
9,Easy Listening,13


### Determine the top-selling genres in countries other than the USA and identify any commonalities or differences.


In [110]:

query30 = pd.read_sql("""
SELECT
 c.Country,
 g.Name AS GenreName,
 SUM(il.Quantity) AS TotalSales
FROM
 Genre AS g
 JOIN Track AS t ON g.Genre_Id = t.Genre_Id
 JOIN Invoice_Line AS il ON t.Track_Id = il.Track_Id
 JOIN Invoice AS i ON il.Invoice_Id = i.Invoice_Id
 JOIN Customer AS c ON i.Customer_Id = c.Customer_Id
WHERE
 c.Country <> 'USA' -- Exclude the USA
GROUP BY
 c.Country,g.name
 order by 3 desc
""", conn)
query30

,country,GenreName,TotalSales
0,Canada,Rock,333
1,France,Rock,211
2,Brazil,Rock,205
3,Germany,Rock,194
4,United Kingdom,Rock,166
...,...,...,...
252,Sweden,Reggae,1
253,Sweden,Soundtrack,1
254,United Kingdom,Electronica/Dance,1
255,United Kingdom,Heavy Metal,1
